In [1]:
# Author: Eric Chandler <echandler@uchicago.edu>
# Description: Trying to write some computer code to make sense of the Durlauf-Seshadri paper

## Goals
Implement the Durlauf-Seshadri model as a simulation, reproduce the expected stylized facts, identify causal mechanism, test implications of extending model or relaxing certain assumptions.
## 
The paper's main arguments are 1) segregation causes and aggravates inequality 2) and that inequality decreases subsequent generations' mobility.
## Stylized Facts
* intertemporal great gadby curve emerges
* permanent segregation without explicit poverty/affluence traps
## Model Structure
* income is certain and depends on child human capital
* only two periods of life (child/get capital, adult/get income)
* local public finance of schools
* students in same school gain same amount of capital from school
* parents prefer more affluent, larger neighborhoods
* inequality increases segregation

## Key Assumptions
* local public finance of schools
* no marriage or inter-family income sharing
* one child, no elders, no population growth or shrink
* no direct parental investment (only via neighborhood)
* parental utility depends on child expected income, not on child expected utility or human capital
* no credit constraint
## Further Directions
* alternative mechanisms for mobility:
    * credit constraints
    * productivity of parent capital investment in child depends on parent capital level
    * other ways for heterogenous AR
* peer effects in capital development
* role model effects in capital development




## Model
cross-sectional inequality -> neighborhood differences

segregation:= increasing function of cross-sectional inequality

mobility := parental input + neighborhood quality

agents in firm/school split earninge evenly. no additional benefits for most productive person.

### Optional
* assortive matching of workers by skill because its more efficient overall
* assortive matching of students to schools because its more efficient overall
* spatial segregation




In [3]:
import numpy as np
import pandas as pd
from numpy.random import default_rng
import scipy

## Define Model Processes

In [ ]:
# Parameters
N_FAMILIES = 100
N_TIMESTEPS = 100
N_NEIGHBORHOODS = 2
INCOME_GROWTH = 0   # alpha in eq(1)
INCOME_MOBILITY_COEF = 0  # beta in eq(1)
INCOME_NOISE_AUTOREG = 1 
TAX_RATE = .1
rng = default_rng(seed=12345)

In [ ]:
def generate_white_noise(rng, size, mean=0, var=1):
    """returns `size` random samples"""
    # any white noise process will do
    return rng.standard_normal(mean, var, size)

def generate_income_noise(rng, parent_noise=None):
    """aka epsilon, a MA(1) process.  from eq (1)"""
    noise = generate_white_noise(rng, N_FAMILIES)
    if parent_noise is not None:
        noise += INCOME_NOISE_AUTOREG * parent_noise
    return noise
    
def generate_mobility(parent_income):
    """ beta in eq(1) and eq(2) """
    # follows eq(1) for now
    return np.full(parent_income.shape, INCOME_MOBILITY_COEF)

def transmit_income(rng, parent_income, parent_noise, child_capital):
    """one intergenerational timestep. eq(1) """
    mobility = generate_mobility(parent_income)
    noise = generate_income_noise(rng, parent_noise)
    offspring_income = INCOME_GROWTH + mobility * parent_income + noise
    return offspring_income, noise

def initialize_income(rng):
    """ create initial income distribution """
    # this is arbitrary right now
    income = np.full(N_FAMILIES, 1)
    return income, np.zeros(income.shape)

def initialize_human_capital(rng):
    """ create initial human capital distribution """
    # this is arbitrary right now
    hc = np.full(N_FAMILIES, 1)
    return hc

def initialize_neighborhoods(rng):
    """ create initial neighborhood selection """
    # this is arbitrary right now
    neighborhoods = np.full(N_FAMILIES, 0)
    neighborhoods[0:N_FAMILIES/2] = 1
    return neighborhoods

def pick_neighborhood(rng, adult_income, child_neighborhood):
    """ select into neighborhoods """
    # this is arbitrary now. should use differential selection
    return child_neighborhood

def pay_taxes(adult_income, adult_neighborhood):
    revenue = np.zeros(N_NEIGHBORHOODS)
    for n in range(N_NEIGHBORHOODS):
        taxes = adult_income[adult_neighborhood == n] * TAX_RATE
        adult_income[adult_neighborhood == n] -= taxes
        revenue[n] = taxes.sum()
    return revenue

def receive_human_capital(parent_neighborhood, taxbase):
    hc = np.zeros(N_FAMILIES)
    for n in range(N_NEIGHBORHOODS):
        n_size = (parent_neighborhood == n).sum()
        hc[parent_neighborhood == n] = taxbase[n] / n_size
    return hc

def run():
    income = np.zeros(N_TIMESTEPS, N_FAMILIES)
    neighborhoods = np.zeros(N_TIMESTEPS, N_FAMILIES)
    human_capital = np.zeros(N_TIMESTEPS, N_FAMILIES)
    taxbase = np.zeros(N_NEIGHBORHOODS)
    income[0, :], income_noise = initialize_income(rng)
    neighborhoods[0, :] = initialize_neighborhoods(rng)
    human_capital[0, :] = initialize_human_capital(rng)

    for t in range(1, N_TIMESTEPS):
        # Adult things
        income[t, :], income_noise = transmit_income(rng, income[t-1, :], income_noise, human_capital[t-1, :])
        neighborhoods[t, :] = pick_neighborhood(rng, income[t-1, :], neighborhoods[t-1, :])
        taxbase = pay_taxes(income[t, :], neighborhoods[t, :])
        # Child things
        human_capital[t, :] = receive_human_capital(neighborhoods[t, :], taxbase)

    pd.DataFrame(income).plot()
                                            